# Internet of Wands (WIP)

This notebook is part of [*Practical Data Science for IoT*](https://github.com/pablodecm/datalab_ml_iot) tutorial by Pablo de Castro

## Overview of Use Case

The aim of this example is to demonstrate an end-to-end
example of a machine learning for a (consumer) IoT application
and remark the main challenges associated.

**The use case chosen is a imaginary application where smartphones
devices act as magic wands and we want to make a spell recognition
system, which will be referred as Internet of Wands (IoW).**

We will be focussing on how could we can collect and process 
for training and evaluating a model for such an application. We will also
discuss how we could deploy to production.


### Important Remark

Most of the discussion and technology choices to follow are
not unique of this application and could be readily applied
to use cases  such as:
- Human Activity Recognition with Wearables (e.g. running, lying down, driving or sitting)
- Elderly fall/accident/emergency alert system
- Possible Consumer Applications, for example:
    - Gym Repetition Counter: identify exercise and count the reps based on wearables
    - Parkinson Disease Early Detection system
- Also broadly related with distributed training applications such a self-driving cars (e.g. Tesla Object recognition model [[1]](#References))


## Important Aspects

We will be discussing many aspects of the data cycle in supervised ML workflows for IoT, such as:

- *Training Data Collection*:
    - What device/hardware/configuration are we gonna use for a given application?
    - Which sensors and additional data are relevant?
    - Who/how is gonna be labelling/labelled the data? Do they need training to standarise the process?
    - How much data we need for the application?
    - Can we oversee and control the data collection process?
    - How can we make data labelling it as easy as possible?
    - Can we replicate the training conditions in production?
    - **How expensive is it gonna be?**

- *Training Data Transport and Processing*:
    - Where and how are we gonna store the data?
    - How are we gonna transfer data from the devices to our data processing center?
    - How much preprocessing we are gonna do on the device (i.e. edge computing)?
    - How can we ensure security and privacy (e.g. transport encryption)?
    - Have we test the data collection framework properly before data collection starts?
    - **What volume of data is expected to flow to the servers per unit of time? Will the infrastructure scale and be robust enough?**

- *Data analysis and model building*:
    - What do we want to do?
    - Which tools/platforms/servers are we gonna use to explore the data?
    - What type of data are we studying (e.g. time-series sensor, audio, images, text, etc)?
    - What is the dimensionality and structure of the data?
    - What are the possible factors that affect to the variance of
      the data (e.g. data collection issues or changes in the
      environment)?
    - How easy will it be?
    - Which techniques are more appropiate for a given type and volume of data?
    - Can we complement with existing datasets or starting from a pre-trained model?
    - How are we gonna to evaluate the performance to have unbiased measures?
    - **What is the right trade-off between model complexity and
    performance for the given application?**



- *Production Environment*:
    - How are we gonna be using the resulting model in production?
    - Can we deploy it first as a beta or internally to verify that it works as expected?
    - Where are we gonna to carry out the model evaluation (e.g. our own remote servers, cloud or device)?
    - Can we setup a loop monitoring and redeployment the model in production?
    - **How much is expected to be gained by training with more data or improving the model?**

## Data Collection Infrastructure

Here is an scheme of how the IOW data collection infrastructure,
that uses common IoT technologies (e.g. a MQTT broker and node-red):

<div align="center">
  <img src="https://raw.githubusercontent.com/pablodecm/datalab_ml_iot/master/04_internet_of_wands/images/iow_infrastructure.png" height="50%" style="max-width: 80%">
</div>

## Data Collection Campaign

Go with your smartphone to https://iow.pablodecm.com/ and generate magic spell demostrations.



## Downloading And Loading Latest Dataset

You download the latest dataset from the GitHub repository.

In [ ]:
!wget "https://github.com/pablodecm/datalab_ml_iot/raw/master/04_internet_of_wands/iow_data.zip"; unzip -o iow_data.zip

### Loading the Data

We have do decide how we want to represent the data and also
work on a custom reader for our set of json-based files.

In [ ]:
cat iow_data/alohomora/Cedric_2579103d.01bba.json

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

example_file = "iow_data/wingardium-leviosa/Peppapig_9b2bd7a9.0696f8.json"

example_df = pd.read_json(example_file)
print(example_df.dtypes)
example_df

In [ ]:
# we can apply a series transformation and then invert to get it in a much better format
accel_df = example_df.accel_data.apply(pd.Series).T
accel_df["timestamp"] = pd.to_timedelta(accel_df.timestamp, unit="ms")
accel_df

In [ ]:
# timestamps of giro and accel sensors are high precision with a common origin
gyro_df = example_df.gyro_data.apply(pd.Series).T
gyro_df["timestamp"] = pd.to_timedelta(gyro_df.timestamp, unit="ms")
gyro_df


In [ ]:
# we can use the minimum of timestamp as origin to have the same reference
min_timestamp = min([accel_df.timestamp.min(),gyro_df.timestamp.min()])

accel_df["timestamp"] -= min_timestamp
gyro_df["timestamp"] -= min_timestamp
# set first element to 0 to have the same start
accel_df.loc[0,"timestamp"] = pd.Timedelta(0)
gyro_df.loc[0,"timestamp"] = pd.Timedelta(0)
accel_df = accel_df.set_index("timestamp")
gyro_df = gyro_df.set_index("timestamp")

accel_df.head()

In [ ]:
# still the sampling is slightly different
gyro_df.head()

In [ ]:
# we can resample to fix that
resampled_accel_df = accel_df.resample("20ms").mean().interpolate('time')
resampled_gyro_df = gyro_df.resample("20ms").mean().interpolate('time')

resampled_accel_df

In [ ]:
# we can then finally merge
merged_df = pd.merge(resampled_accel_df, resampled_gyro_df, 
                     left_index=True, right_index=True,
                     suffixes=["_accel","_gyro"], how="inner")

merged_df

In [ ]:
fig, axs = plt.subplots(2, figsize=(12,12))

merged_df.filter(regex="._accel").plot(ax=axs[0])
merged_df.filter(regex="._gyro").plot(ax=axs[1])

In [ ]:
# we can define a function to do all the pre-processing for a given json file

from pathlib import Path

def load_df_from_iow_json(json_path: Path) -> pd.DataFrame:
    
    # load json file to dataframe
    raw_df = pd.read_json(json_path)
    
    # convert to series to avoid having object elements
    accel_df = raw_df.accel_data.apply(pd.Series).T
    accel_df["timestamp"] = pd.to_timedelta(accel_df.timestamp, unit="ms")
    gyro_df = raw_df.gyro_data.apply(pd.Series).T
    gyro_df["timestamp"] = pd.to_timedelta(gyro_df.timestamp, unit="ms")
    
    # we can use the minimum of timestamp as origin to have the same reference
    min_timestamp = min([accel_df.timestamp.min(),gyro_df.timestamp.min()])

    # set the same timestamp origin
    accel_df["timestamp"] -= min_timestamp
    gyro_df["timestamp"] -= min_timestamp
    # set first element to 0 to have the same start
    accel_df.loc[0,"timestamp"] = pd.Timedelta(0)
    gyro_df.loc[0,"timestamp"] = pd.Timedelta(0)
    accel_df = accel_df.set_index("timestamp")
    gyro_df = gyro_df.set_index("timestamp")
    
    # resample and interpolate to make homogeneous
    resampled_accel_df = accel_df.resample("20ms").mean().interpolate('time')
    resampled_gyro_df = gyro_df.resample("20ms").mean().interpolate('time')
    
    # merge in a single df
    merged_df = pd.merge(resampled_accel_df, resampled_gyro_df, 
                         left_index=True, right_index=True,
                         suffixes=["_accel","_gyro"], how="inner")
    
    del raw_df["accel_data"]
    del raw_df["gyro_data"]
    
    return merged_df, raw_df.loc["timestamp"].copy()


In [ ]:
# check if it works from example
load_df_from_iow_json(example_file)[0]

In [ ]:
md_fields = ["spell_select","device_select","wizard_name"]
data_path = Path("./iow_data")

merged_df_dict = {}
for path in data_path.glob("*/*"):
    rel_path = path.relative_to(data_path)
    merged_df, metadata = load_df_from_iow_json(path)
    
    spell_id = (rel_path.parent/rel_path.stem).as_posix()
    key = tuple(metadata.loc[md_fields].to_list() + [spell_id])
    merged_df_dict[key] = merged_df


In [ ]:
all_df = pd.concat(merged_df_dict, names = (md_fields+ ["spell_id"]))
all_df

In [ ]:
test_subset = ['pablodecm']
test_df = all_df.loc[(slice(None),slice(None),list(test_subset),slice(None)),:]
all_others = set(all_df.index.get_level_values(2).unique()) - set(test_subset)
train_df = all_df.loc[(slice(None),slice(None),list(all_others),slice(None)),:]

In [ ]:
train_df

In [ ]:
# try to create a spell classifier based on this data

About high-resolution time in web applications:

https://www.w3.org/TR/hr-time-2/#dfn-time-origin

## References

For an overview of a state-of-the-art distributed training
infrastructure including redeployment and the importance
of edge in real-time applications you can check the Tesla Autonomy day presentation:

- [1] [*Tesla Autonomy Day*](https://www.youtube.com/watch?v=Ucp0TTmvqOE)  Youtube video (+2hrs)

There are several publications using combinations of RNNs and CNNs
for dealing with IoT sensor data, for task such as
Human Activity Recognition:
- [2] Yao, Shuochao, et al. [*Deepsense: A unified deep learning framework for time-series mobile sensing data processing*](https://arxiv.org/abs/1611.01942) Proceedings of the 26th International Conference on World Wide Web. International World Wide Web Conferences Steering Committee, 2017.